
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

### Install required libraries

In [1]:
!pip install numpy==1.16.4

     |████████████████████████████████| 17.3MB 1.2MB/s 
ERROR: umap-learn 0.4.4 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [1]:
!pip install tensorflow==1.12
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 

!pip install tf_slim
!pip install wget

import os
import wget

!git clone https://github.com/tensorflow/models.git && (cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc)
!rsync -avzh --progress /content/models /content/tensorflow/
!rm -rf /content/models

os.environ['PYTHONPATH'] += ':/content/tensorflow/models/research/:/content/tensorflow/models/research/slim'

%cd {BASE_DIR}/models/research
!protoc object_detection/protos/*.proto --python_out=.
!python setup.py build
!python setup.py install

     |████████████████████████████████| 83.1MB 69kB/s 
     |████████████████████████████████| 3.1MB 44.4MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
--2020-06-11 21:15:39--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5034 (4.9K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   4.92K  --.-KB/s    in 0s      

2020-06-11 21:15:40 (69.8 MB/s) - ‘constants.py’ saved [5034/5034]



#### Download model config and save to to config directory

In [7]:
wget.download(pretrained_model_key['config_url'],CONFIG_DIR)

'/content/tensorflow/workspace/Face-Mask-Detection/training/config/ssd_mobilenet_v2_quantized_300x300.config'

### Prepare dataset ###

#### Connect to Google Drive

In [8]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract

In [9]:
!gsutil cp {DRIVE_DEV}model_graph.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_graph.zip -d {INF_GRAPH}

!gsutil cp {DRIVE_DEV}model_checkpoints.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_checkpoints.zip -d {CKPT_DIR}

Copying file:///content/drive/My Drive/Machine Learning/Projects/Face-Mask-Detection/Dev/model_graph.zip...
-
Operation completed over 1 objects/50.6 MiB.                                     
Archive:  /content/tensorflow/workspace/model_graph.zip
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/checkpoint  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/pipeline.config  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/saved_model.pb  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph

### Export inference graph

Use this option if you want to convert to TFjs later

In [0]:
!python {BASE_DIR}/models/research/object_detection/export_inference_graph.py \
  --pipeline_config_path="{CONFIG_DIR}/{pretrained_model_key['name']}.config" \
  --trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-18560" \
  --output_directory="{INF_GRAPH}" \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0611 10:44:29.284205 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_inference_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0611 10:44:29.293980 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:348: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0611 10:44:29.294343 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:113: The name tf.placeholder is deprecated. Please 

### Export TF Lite SSD inference graph

Use this option for TF Lite conversion

In [10]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="{CONFIG_DIR}/{pretrained_model_key['name']}.config" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-19611" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

2020-06-11 21:21:50.333863: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-11 21:21:54.593521: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying strip_unused_nodes


### Convert graph to TF Lite

In [11]:
!tflite_convert \
--output_file="{TF_LITE_MODEL}/detect2.tflite" \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--inference_type=QUANTIZED_UINT8 \
--input_arrays="{INPUT_TENSORS}" \
--output_arrays="{OUTPUT_TENSORS}"  \
--mean_values=128 \
--std_dev_values=128 \
--input_shapes=1,300,300,3 \
--change_concat_input_ranges=false \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops \
--output_format=TFLITE \
--default_ranges_min=0 \
--default_ranges_max=255

2020-06-11 21:22:11.079323: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


## Save 

#### Archive the inference graph

In [0]:
!zip -r inf-graphv1.zip {INF_GRAPH}
!gsutil cp inf-graphv1.zip {DRIVE_DEV}

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



#### Archive the TF Lite graph

In [11]:
!zip -r tflite-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp tflite-graphv1.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pb (deflated 8%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pbtxt (deflated 56%)
Copying file://tflite-graphv1.zip...
-
Operation completed over 1 objects/39.5 MiB.                                     


#### Archive the TF Lite model

In [12]:
!zip -r tflite-modelv4.zip {TF_LITE_MODEL}
!gsutil cp tflite-modelv4.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/detect2.tflite (deflated 39%)
Copying file://tflite-modelv4.zip...
/ [1 files][  2.8 MiB/  2.8 MiB]                                                
Operation completed over 1 objects/2.8 MiB.                                      
